In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import optuna
import random
from tqdm.notebook import tqdm

from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.feature_selection import RFECV, SelectKBest, f_classif
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from concurrent.futures import ThreadPoolExecutor

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from pprint import pprint
import os

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

experiment_name = 'multi-models_with_original_dataset'

In [2]:
train = pd.read_csv('train.csv')
original = pd.read_csv('Fault.csv', delimiter='\t')
test = pd.read_csv('test.csv')

In [3]:
train.shape, original.shape, test.shape

((19219, 35), (1940, 34), (12814, 28))

In [4]:
# Remove id column and check the dataset
train.drop('id', axis=1, inplace=True)
train.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [5]:
test.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,19219,1015,1033,3826564,3826588,659,23,46,62357,67,127,1656,0,1,150,0.3877,0.4896,0.3273,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,19220,1257,1271,419960,419973,370,26,28,39293,92,132,1354,0,1,40,0.1629,0.4136,0.0938,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,19221,1358,1372,117715,117724,289,36,32,29386,101,134,1360,0,1,40,0.0609,0.6234,0.4762,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,19222,158,168,232415,232440,80,10,11,8586,107,140,1690,1,0,100,0.4439,0.3333,0.8182,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,19223,559,592,544375,544389,140,19,15,15524,103,134,1688,1,0,60,0.8191,0.2619,0.4286,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170


In [6]:
original_dataset_cols = list(train.columns)

In [7]:
features_list = ['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
                 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
                 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
                 'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
                 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
                 'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
                 'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
                 'SigmoidOfAreas']

In [8]:
original.columns = original_dataset_cols
original.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
1,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
2,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
3,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
4,430,441,100250,100337,630,20,87,62357,64,127,1387,0,1,40,0.6200,0.3417,0.1264,0.0079,0.5500,1.0000,1.0,2.7993,1.0414,1.9395,0.8736,-0.2267,0.9874,1,0,0,0,0,0,0


In [9]:
# Concat the train and original dataset
combined_df = pd.concat([train, original], axis=0).reset_index(drop=True)

In [10]:
# Got function from https://www.kaggle.com/code/thomasmeiner/ps4e3-eda-feature-engineering-model

def reformat_data(df: pd.DataFrame) -> pd.DataFrame:
    target_cols = [
        "Pastry", #4
        "Z_Scratch", #6
        "K_Scatch", #2
        "Stains", #5
        "Dirtiness", #1
        "Bumps", #0
        "Other_Faults", #3
    ]
    non_target_cols = df.drop(target_cols, axis=1).columns.to_list()
    
    binary_dfs = []
    
    for col in target_cols:
        temp_df = df.loc[:, non_target_cols + [col]]
        temp_df = temp_df.loc[temp_df[col] == 1].copy() # keep positives only
        temp_df[col] = col # target value is class name now
        temp_df = temp_df.rename(columns={col: "target"}) # make target col name uniform for final concat
        binary_dfs.append(temp_df)
        
    reformatted_df = pd.concat(binary_dfs)
    return reformatted_df

In [11]:
# def reformat_data(df: pd.DataFrame) -> pd.DataFrame:
#     target_cols = [
#         "Pastry",
#         "Z_Scratch",
#         "K_Scatch",
#         "Stains",
#         "Dirtiness",
#         "Bumps",
#         "Other_Faults",
#     ]
#     non_target_cols = df.drop(target_cols, axis=1).columns.to_list()

#     binary_dfs = []

#     for col in target_cols:
#         temp_df = df.loc[:, non_target_cols + [col]]
#         temp_df = temp_df.loc[temp_df[col] == 1].copy() # sub sample to lowest class
#         temp_df[col] = col # target value is class name now
#         temp_df = temp_df.rename(columns={col: "target"}) # make target col name uniform for final concat
#         binary_dfs.append(temp_df)

#     # collect non_defect rows
#     temp_df = df.loc[
#         (df["Pastry"] == 0) &
#         (df["Z_Scratch"] == 0) &
#         (df["K_Scatch"] == 0) &
#         (df["Stains"] == 0) &
#         (df["Dirtiness"] == 0) &
#         (df["Bumps"] == 0) &
#         (df["Other_Faults"] == 0)
#     ]
#     temp_df = temp_df.loc[: , non_target_cols]
#     temp_df["target"] = "No defect"
#     binary_dfs.append(temp_df)

#     reformatted_df = pd.concat(binary_dfs)
#     return reformatted_df

In [12]:
combined_df = reformat_data(combined_df)
combined_df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,target
7,1673,1687,294065,294091,571,38,57,53142,77,110,1692,0,1,300,0.1491,0.4326,0.9643,0.0142,0.5686,0.7179,1.0,2.7528,1.3802,1.7559,0.0357,-0.2661,0.9408,Pastry
24,1538,1549,849219,849235,275,19,32,28986,71,117,1626,1,0,70,0.1494,0.3990,0.3750,0.0088,0.6316,1.0000,1.0,2.4393,1.0792,1.5051,0.6250,-0.2988,0.6330,Pastry
35,1310,1316,435871,435916,153,16,32,17101,104,132,1352,0,1,40,0.0532,0.3854,0.3333,0.0044,0.3750,0.9688,1.0,2.1847,0.7782,1.5051,0.6667,-0.0916,0.4025,Pastry
60,765,774,6571361,6571375,59,9,12,6682,77,133,1360,0,1,100,0.3613,0.3571,0.2800,0.0052,0.7778,1.0000,1.0,1.7708,0.8451,1.0792,0.7200,-0.1522,0.1892,Pastry
67,1677,1686,1319063,1319076,91,10,15,5608,57,95,1692,1,0,70,0.0024,0.3583,0.6667,0.0047,0.8000,1.0000,1.0,1.9590,0.9031,1.1761,0.3333,-0.3868,0.2660,Pastry


In [13]:
TARGET = 'target'

In [14]:
label_encoder = LabelEncoder()

label_encoder.fit(combined_df[TARGET])

combined_df[TARGET] = label_encoder.transform(combined_df[TARGET])

combined_df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,target
7,1673,1687,294065,294091,571,38,57,53142,77,110,1692,0,1,300,0.1491,0.4326,0.9643,0.0142,0.5686,0.7179,1.0,2.7528,1.3802,1.7559,0.0357,-0.2661,0.9408,4
24,1538,1549,849219,849235,275,19,32,28986,71,117,1626,1,0,70,0.1494,0.3990,0.3750,0.0088,0.6316,1.0000,1.0,2.4393,1.0792,1.5051,0.6250,-0.2988,0.6330,4
35,1310,1316,435871,435916,153,16,32,17101,104,132,1352,0,1,40,0.0532,0.3854,0.3333,0.0044,0.3750,0.9688,1.0,2.1847,0.7782,1.5051,0.6667,-0.0916,0.4025,4
60,765,774,6571361,6571375,59,9,12,6682,77,133,1360,0,1,100,0.3613,0.3571,0.2800,0.0052,0.7778,1.0000,1.0,1.7708,0.8451,1.0792,0.7200,-0.1522,0.1892,4
67,1677,1686,1319063,1319076,91,10,15,5608,57,95,1692,1,0,70,0.0024,0.3583,0.6667,0.0047,0.8000,1.0000,1.0,1.9590,0.9031,1.1761,0.3333,-0.3868,0.2660,4


In [15]:
combined_df.shape, test.shape

((20362, 28), (12814, 28))

In [16]:
X = combined_df.drop([TARGET], axis=1)
y = combined_df[TARGET]

n_splits = 3
sk10 = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=5)

In [23]:
lgbm_params_1 = {'objective': 'multiclass', 'num_class': 7, 'n_jobs': -1, 'random_state': 5, 'class_weight': None, 'boosting': 'gbdt', 'colsample_bytree': 0.3732513300418112, 'learning_rate': 0.11407449915410499, 'max_depth': 41, 'min_child_samples': 46, 'min_child_weight': 5.402556610174332, 'min_split_gain': 0.7031617243606461, 'n_estimators': 168, 'num_leaves': 4, 'reg_alpha': 0.6688125857557925, 'reg_lambda': 0.3006431833867248, 'subsample': 0.7700917000679017}
lgbm_params_2 = {'objective': 'multiclass', 'num_class': 7, 'n_jobs': -1, 'random_state': 5, 'class_weight': None, 'boosting': 'dart', 'colsample_bytree': 0.5772149549047157, 'learning_rate': 0.33002721528421214, 'max_depth': 28, 'min_child_samples': 23, 'min_child_weight': 7.99839845350044, 'min_split_gain': 0.9239893181066746, 'n_estimators': 699, 'num_leaves': 21, 'reg_alpha': 0.4760265812563772, 'reg_lambda': 0.15634739914542023, 'subsample': 0.4590800039755062}
lgbm_params_3 = {'objective': 'multiclass', 'num_class': 7, 'n_jobs': -1, 'random_state': 5, 'class_weight': None, 'boosting': 'dart', 'colsample_bytree': 0.31193732162008925, 'learning_rate': 0.03107980185406725, 'max_depth': 58, 'min_child_samples': 26, 'min_child_weight': 0.9044244579504741, 'min_split_gain': 0.6988647291560484, 'n_estimators': 814, 'num_leaves': 10, 'reg_alpha': 0.8952949360981323, 'reg_lambda': 0.12911118443546726, 'subsample': 0.9423432553265046}
lgbm_params_4 = {'objective': 'multiclass', 'num_class': 7, 'n_jobs': -1, 'random_state': 5, 'class_weight': None, 'boosting': 'dart', 'colsample_bytree': 0.35885033102300595, 'learning_rate': 0.21902422874173558, 'max_depth': 3, 'min_child_samples': 34, 'min_child_weight': 4.768878441321062, 'min_split_gain': 0.8804385788228134, 'n_estimators': 334, 'num_leaves': 19, 'reg_alpha': 0.9922927799932719, 'reg_lambda': 0.010366800790489683, 'subsample': 0.5713199333429264}
hist_params_1 = {'random_state': 5, 'learning_rate': 0.052466626497163174, 'max_iter': 72, 'max_leaf_nodes': 23, 'min_samples_leaf': 66, 'l2_regularization': 0.8935374409410728, 'max_bins': 151, 'max_depth': 45}
hist_params_2 = {'random_state': 5, 'learning_rate': 0.08690147432171641, 'max_iter': 363, 'max_leaf_nodes': 49, 'min_samples_leaf': 77, 'l2_regularization': 0.5540219768909869, 'max_bins': 175, 'max_depth': 4}
hist_params_3 = {'random_state': 5, 'learning_rate': 0.09432387429473288, 'max_iter': 269, 'max_leaf_nodes': 22, 'min_samples_leaf': 87, 'l2_regularization': 0.9939707101523615, 'max_bins': 246, 'max_depth': 39}
hist_params_4 = {'random_state': 5, 'learning_rate': 0.010334329050417028, 'max_iter': 450, 'max_leaf_nodes': 20, 'min_samples_leaf': 87, 'l2_regularization': 0.6496937198849339, 'max_bins': 118, 'max_depth': 63}
hist_params_5 = {'random_state': 5, 'learning_rate': 0.05116790153455628, 'max_leaf_nodes': 15, 'min_samples_leaf': 358, 'l2_regularization': 0.870263366116972, 'max_bins': 78}
extrat_params_1 = {'n_jobs': -1, 'random_state': 5, 'max_leaf_nodes': 1245, 'min_samples_leaf': 2, 'min_samples_split': 18, 'min_weight_fraction_leaf': 0.0001389853877993949, 'n_estimators': 2712}
extrat_params_2 = {'n_jobs': -1, 'random_state': 5, 'max_leaf_nodes': 1643, 'min_samples_leaf': 2, 'min_samples_split': 75, 'min_weight_fraction_leaf': 4.1528306236372726e-05, 'n_estimators': 2580}
extrat_params_3 = {'n_jobs': -1, 'random_state': 5, 'max_leaf_nodes': 2768, 'min_samples_leaf': 1, 'min_samples_split': 55, 'min_weight_fraction_leaf': 5.532962720126739e-05, 'n_estimators': 2989}
extrat_params_4 = {'n_jobs': -1, 'random_state': 5, 'max_leaf_nodes': 1835, 'min_samples_leaf': 1, 'min_samples_split': 50, 'min_weight_fraction_leaf': 4.0520720198360216e-05, 'n_estimators': 2504}
rf_params_1 = {'max_features': 'log2', 'criterion': 'entropy', 'n_estimators': 1000, 'random_state': 5, 'n_jobs': -1, 'max_depth': 28, 'max_leaf_nodes': 532, 'max_samples': 0.997319110588176, 'min_samples_leaf': 1, 'min_samples_split': 15}
rf_params_2 = {'max_features': 'log2', 'criterion': 'entropy', 'n_estimators': 1000, 'random_state': 5, 'n_jobs': -1, 'max_depth': 23, 'max_leaf_nodes': 692, 'max_samples': 0.7633473827420749, 'min_samples_leaf': 4, 'min_samples_split': 7}
# rf_params_3 = 
# rf_params_4 = 
cat_params_1 = {
'bagging_temperature' : 0.5,
'colsample_bylevel'   : 0.50,
'iterations'          : 2500,
'learning_rate'       : 0.04,
'max_depth'           : 8,
'l2_leaf_reg'         : 1.235,
'min_data_in_leaf'    : 25,
'random_strength'     : 0.35, 
'max_bin'             : 160,
'verbose'             : False,
'early_stopping_rounds': 100,
'random_state'        : 5}
cat_params_2 = {
'bagging_temperature' : 0.1,
'colsample_bylevel'   : 0.88,
'iterations'          : 3000,
'learning_rate'       : 0.065,
'max_depth'           : 7,
'l2_leaf_reg'         : 1.75,
'min_data_in_leaf'    : 25,
'random_strength'     : 0.1, 
'max_bin'             : 100,
'verbose'             : False,
'early_stopping_rounds': 100,
'random_state'        : 5}
cat_params_3 = {
'bagging_temperature' : 0.2,
'colsample_bylevel'   : 0.85,
'iterations'          : 2500,
'learning_rate'       : 0.025,
'max_depth'           : 7,
'l2_leaf_reg'         : 1.235,
'min_data_in_leaf'    : 8,
'random_strength'     : 0.60, 
'max_bin'             : 160,
'verbose'             : False,
'early_stopping_rounds': 100,
'random_state'        : 5}
cat_params_4 = {
'grow_policy'         : 'Lossguide',
'colsample_bylevel'   : 0.25,
'iterations'          : 2500,
'learning_rate'       : 0.035,
'max_depth'           : 7,
'l2_leaf_reg'         : 1.80,
'random_strength'     : 0.60, 
'max_bin'             : 160,
'verbose'             : False,
'early_stopping_rounds': 100,
'random_state'        : 5}

In [27]:
models = [
    LGBMClassifier(n_jobs=-1, random_state=5),
    LGBMClassifier(**lgbm_params_1),
    LGBMClassifier(**lgbm_params_2),
    LGBMClassifier(**lgbm_params_3),
    LGBMClassifier(**lgbm_params_4),
    XGBClassifier(random_state=5),
    RandomForestClassifier(random_state=5),
    RandomForestClassifier(**rf_params_1),
    RandomForestClassifier(**rf_params_2),
    ExtraTreesClassifier(random_state=5),
    ExtraTreesClassifier(**extrat_params_1),
    ExtraTreesClassifier(**extrat_params_2),
    ExtraTreesClassifier(**extrat_params_3),
    ExtraTreesClassifier(**extrat_params_4),
    HistGradientBoostingClassifier(random_state=5),
    HistGradientBoostingClassifier(**hist_params_1),
    HistGradientBoostingClassifier(**hist_params_2),
    HistGradientBoostingClassifier(**hist_params_3),
    HistGradientBoostingClassifier(**hist_params_4),
    HistGradientBoostingClassifier(**hist_params_5),
    CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100),
    CatBoostClassifier(**cat_params_1),
    CatBoostClassifier(**cat_params_2),
    CatBoostClassifier(**cat_params_3),
    CatBoostClassifier(**cat_params_4),
    ]

In [19]:
def evaluate_models_roc(models, X, y, important_features, cv_split, experiment_name):
    MLA_compare = pd.DataFrame(columns=['MLA Name', 
                                        'MLA Parameters', 
                                        'MLA Train ROC AUC', 
                                        'MLA Test ROC AUC', 
                                        'MLA Test ROC AUC Std', 
                                        'MLA Time'])
    
    def evaluate_model(alg, idx):
        MLA_name = alg.__class__.__name__
        features = important_features.get(MLA_name, [])

        # Check if the list of important features is empty
        if len(features) == 0:
            # If empty, return results with zero values
            print(f'Skipping {MLA_name} due to no important features.')
            return {
                'MLA Name': MLA_name,
                'MLA Parameters': str(alg.get_params()),
                'MLA Train ROC': 0,
                'MLA Test ROC': 0,
                'MLA Test ROC Std': 0,
                'MLA Time': "0 min 0.00 sec",
            }
        
        cv_results = cross_validate(alg, 
                                    X[features], 
                                    y, cv=cv_split, 
                                    scoring='roc_auc_ovr', 
                                    return_train_score=True, 
                                    n_jobs=-1)

        # Time formatting
        mean_fit_time = cv_results['fit_time'].mean()
        minutes, seconds = divmod(mean_fit_time, 60)

        # Results population
        result = {
            'MLA Name': MLA_name,
            'MLA Parameters': str(alg.get_params()),
            'MLA Train ROC AUC': cv_results['train_score'].mean(),
            'MLA Test ROC AUC': cv_results['test_score'].mean(),
            'MLA Test ROC AUC Std': cv_results['test_score'].std(),
            'MLA Time': f"{int(minutes)} min {seconds:.2f} sec",
        }

        print(f'Done with {MLA_name}.')
        return result

    results_list = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(evaluate_model, alg, idx) for idx, alg in enumerate(models)]
        for future in futures:
            result = future.result()
            results_list.append(result)

    MLA_compare = pd.DataFrame(results_list)

    MLA_compare.sort_values(by=['MLA Test ROC AUC'], ascending=False, inplace=True)
    MLA_compare.to_csv(f'{experiment_name}_results.csv', index=False)

    return MLA_compare

In [ ]:
baseline_features = {}

for model in models:
    model_name = model.__class__.__name__

    baseline_features[model_name] = list(X_hist.columns)

In [ ]:
%%time

baseline_models = evaluate_models_roc(models, X, y, baseline_features, sk10, f'{experiment_name}_lgbm')
baseline_models

In [ ]:
%%time

baseline_models = evaluate_models_roc(models, X, y, baseline_features, sk10, f'{experiment_name}')
baseline_models

- Remove Correlated Features

In [ ]:
# Remove correlated features (leaving just 1 of each pair)
# Leave features highly correlated with the target
df_no_corr = X.copy()
correlation_matrix_spear = df_no_corr.corr(method='spearman').abs()

# Select upper triangle of correlation matrix
upper_spear = correlation_matrix_spear.where(np.triu(np.ones(correlation_matrix_spear.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than a threshold (e.g., 0.9 in this case)
to_drop_spear = [column for column in upper_spear.columns if any(upper_spear[column] >= 0.9)]

# Drop features
df_reduced_spear = df_no_corr.drop(to_drop_spear, axis=1)

# Get list of low correlation features excluding TARGET
low_corr_feats_spear = list(df_reduced_spear.columns)

with open('low_corr_spear.txt', 'w') as f:
    f.write(str(low_corr_feats_spear))
    f.write('\n')

# Print the high correlation features effect
# Both pre and post drop dfs contain the TARGET
print(f"Dropped {len(to_drop_spear)} highly correlated features.\nOld Shape of the dataset was {df_no_corr.shape}\nNew shape of the dataset is {df_reduced_spear.shape}")

In [ ]:
%%time

no_corr_features = {}

for model in models:
    model_name = model.__class__.__name__

    no_corr_features[model_name] = list(df_reduced_spear.columns)

In [ ]:
%%time

no_corr_models = evaluate_models_roc(models, df_reduced_spear, y, no_corr_features, sk10, f'{experiment_name}_corr')
no_corr_models

- Feature Importances

In [ ]:
# feat_importance_features = {}

# for model in models:
#     model_name = model.__class__.__name__

#     try:
#         # Initialize array to store feature importances
#         feature_importances = np.zeros(X.shape[1])

#         # Loop through each fold and calculate the feature importances
#         for train_index, test_index in sk10.split(X, y):
#             X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#             y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#             model.fit(X_train, y_train)

#             # Get the feature importances and them to the total
#             feature_importances += model.feature_importances_

#         feature_importances /= n_splits

#         feature_importances_dict = dict(zip(X.columns, feature_importances))

#         df = pd.DataFrame.from_dict(feature_importances_dict, orient='index')

#         # Resetting index with a name for the column
#         df = df.reset_index().rename(columns={'index': 'Feature', 0: 'Avg_Feat_Importance'})
#         df.sort_values(by='Avg_Feat_Importance', ascending=False, inplace=True)

#         # Save to CSV
#         df.to_csv(f'{model_name}_feature_importances.csv')

#         fi_threshold = 0

#         fi_feats = df[df['Avg_Feat_Importance'] > fi_threshold]['Feature'].tolist()

#         feat_importance_features[model_name] = fi_feats
#         print(f'Done with {model_name}')

#     except AttributeError:
#         feat_importance_features[model_name] = list(X.columns)
#         print(f'{model_name} does not have feature_importances_')

In [ ]:
# with open('featimp_features.txt', mode='w') as f:
#     pprint(feat_importance_features, stream=f)

- Permutation Importance

In [ ]:
# Generate a random feature for X
np.random.seed(5)
df_reduced_spear['random_control_feature'] = np.round(np.random.uniform(-2, 2, df_reduced_spear.shape[0]), 6)
df_reduced_spear.shape

In [ ]:
%%time

perm_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

perm_importances = {model.__class__.__name__: [] for model in models}

for i, (train_idx, test_idx) in enumerate(perm_cv.split(df_reduced_spear, y)):
    X_train, X_test = df_reduced_spear.iloc[train_idx], df_reduced_spear.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y_train)
        # Calculate permutation importance
        result = permutation_importance(model, X_test, y_test, n_repeats=5, random_state=5, n_jobs=-1, scoring='roc_auc_ovr')
        perm_importances[model_name].append(result.importances_mean)
        print(f'Done with {model_name}.')
    
    print(f'Done with Fold {i+1}', end='\n\n')

In [ ]:
%%time

# Average importances across folds and export to CSV
for model_name, importances in perm_importances.items():
    avg_importance = np.mean(importances, axis=0)
    importance_df = pd.DataFrame({'Feature': df_reduced_spear.columns, 'Importance': avg_importance})
    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    # Export to CSV
    importance_df.to_csv(f'.\permutation_importances\{model_name}_permutation_importance.csv', index=False)

print('Done with Permuation Importances', end='\n\n')

In [ ]:
directory = 'permutation_importances'

# Initialize a dictionary for the features
perm_important_features = {}

for model in models:
    model_name = model.__class__.__name__
    csv_path = os.path.join(directory, f'{model_name}_permutation_importance.csv')
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)

        # Check for 'random_control_feature' and its importance
        if 'random_control_feature' in df['Feature'].values:
            random_feature_importance = df.loc[df['Feature'] == 'random_control_feature', 'Importance'].iloc[0]
        else:
            random_feature_importance = 0

        # Determine the threshold
        threshold = max(0, random_feature_importance)

        # Filter features where importance is greater than 0
        important_feats_filtered = df[df['Importance'] > threshold]['Feature'].tolist()

        # Reorder important_feats based on the predefined features_list
        important_feats_ordered = [feat for feat in features_list if feat in important_feats_filtered]

        # Add to importance dictionary
        perm_important_features[model_name] = important_feats_ordered

    else:
        print(f'CSV file for {model_name} not found.')

print('Done getting important features dictionary')

In [ ]:
with open('perm_important_features_lgbm.txt', mode='w') as f:
    pprint(perm_important_features, stream=f)

In [ ]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

perm_importance_models = evaluate_models_roc(models, X, y, perm_important_features, sk10, f'{experiment_name}_permimp')
perm_importance_models

- SelectKBest with f_classif

In [ ]:
best_features_list = []
kbest_features = {}

for model in models:
    model_name = model.__class__.__name__

    # Select whichever one had a better CV score generally
    # Also, consider computational expense and accuracy balance
    
    features = perm_important_features[model_name]
    # features = list(df_reduced_spear.columns)

    # incase there is no feature that had importance, go to the next model
    if len(features) == 0:
        continue
	
    X_kbest = X[features]
    best_score = 0
    best_k = 0
    best_features = []

    # Iterate over k from 1 to number of features
    for k in range(1, len(features) + 1):
        print(f'currently running {k} features on {model_name}')
        # Apply SelectKBest
        selector = SelectKBest(f_classif, k=k)
        X_new = selector.fit_transform(X_kbest, y)

        # Get the selected feature names
        selected_features = X_kbest.columns[selector.get_support()]

        # Evaluate the model
        # model = LGBMClassifier(n_jobs=-1, random_state=5)
        roc_auc_scores = cross_validate(model, X_new, y, cv=sk10, scoring='roc_auc_ovr', n_jobs=-1)
        mean_roc_auc_scores = roc_auc_scores['test_score'].mean()

        if mean_roc_auc_scores > best_score:
            best_k = k
            best_score = mean_roc_auc_scores
            best_features = list(selected_features)

    best_features_list.append({'k': best_k,
                    'Selected Features': best_features,
                    'ROC AUC Score': best_score,
                    'Model Name': model_name})
    
    kbest_features[model_name] = best_features

best_features_df = pd.DataFrame(best_features_list)

best_features_df.sort_values(by='ROC AUC Score', ascending=False, inplace=True)

In [ ]:
with open('kbest_features.txt', mode='w') as f:
    pprint(kbest_features, stream=f)

In [ ]:
best_features_df

- RFECV

In [ ]:
%%time

# Initialize empty dictionary for RFECV features
rfecv_features = {}

for alg in models:
    # set name
    MLA_name = alg.__class__.__name__
		
    features = perm_important_features[MLA_name]

    # incase there is no feature that had importance, go to the next model
    if len(features) == 0:
        continue
	
    X_rfecv = X[features]

    try:
        print(f'Starting with {MLA_name}')
        # Create the RFECV object and rank each feature
        selector = RFECV(alg, cv=sk10, step=1, scoring='roc_auc_ovr', verbose=2)
        selector = selector.fit(X_rfecv, y)

        selected_features = list(X_rfecv.columns[selector.support_])

        # Reorder selected_features based on the predefined features_list
        selected_features_ordered = [feat for feat in features_list if feat in selected_features]

        rfecv_features[MLA_name] = selected_features_ordered

        print(f'Done with {MLA_name}', end='\n\n')
    
    except ValueError:
        # In case of an error, keep the original order but filtered by features_list
        features_filtered = [feat for feat in features_list if feat in features]
        rfecv_features[MLA_name] = features_filtered
        print(f'{MLA_name} does not have coef_ or feature_importances_', end='\n\n')

In [ ]:
with open('rfecv_features.txt', mode='w') as f:
    pprint(rfecv_features, stream=f)

In [ ]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

rfecv_models = evaluate_models_roc(models, X, y, rfecv_features, sk10, f'{experiment_name}_rfecv')
rfecv_models

- SFS

In [ ]:
%%time

# Initialize empty dictionary for SFS features
sfs_features = {}

for alg in models:
    # set name
    MLA_name = alg.__class__.__name__

    try:
            
        # features = kbest_features[MLA_name]
        # features = feat_importance_features[MLA_name]
        features = rfecv_features[MLA_name]

        # incase there is no feature that had importance, go to the next model
        if len(features) == 0:
            continue
        
        X_sfs = X[features]

        print(f'Running backward feature selection with {MLA_name}')

        sfs = SFS(alg,
            k_features='best',
            forward=False,
            floating=False,
            scoring='roc_auc_ovr',
            verbose=2,
            n_jobs=-1,
            cv=sk10)
        
        sfs = sfs.fit(X_sfs, y)

        # Get the selected features index
        selected_sfs_idx = list(sfs.k_feature_idx_)

        # Get the feature names
        selected_sfs_feats = X_sfs.columns[selected_sfs_idx]

        selected_features = list(selected_sfs_feats)

        # Reorder selected_features based on the predefined features_list
        selected_features_ordered = [feat for feat in features_list if feat in selected_features]

        sfs_features[MLA_name] = selected_features_ordered

        print(f'Done with {MLA_name}', end='\n\n')

    except KeyError:
        print(f'{MLA_name} not in the dictionary.')

In [ ]:
with open('sfs_features.txt', mode='w') as f:
    pprint(sfs_features, stream=f)

In [20]:
sfs_features = {'CatBoostClassifier': ['X_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'Orientation_Index', 'Luminosity_Index'],
 'ExtraTreesClassifier': ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index'],
 'HistGradientBoostingClassifier': ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'Minimum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index'],
 'LGBMClassifier': ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index'],
 'RandomForestClassifier': ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index'],
 'XGBClassifier': ['X_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'Orientation_Index', 'Luminosity_Index']}

In [26]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

sfs_models = evaluate_models_roc(models, X, y, sfs_features, sk10, f'{experiment_name}_sfs')
sfs_models

Done with LGBMClassifier.
Done with RandomForestClassifier.
Done with RandomForestClassifier.
Done with LGBMClassifier.
Done with ExtraTreesClassifier.
Done with LGBMClassifier.
Done with RandomForestClassifier.
Done with XGBClassifier.
Done with ExtraTreesClassifier.
Done with ExtraTreesClassifier.
Done with LGBMClassifier.
Done with ExtraTreesClassifier.
Done with ExtraTreesClassifier.
Done with HistGradientBoostingClassifier.
Done with HistGradientBoostingClassifier.
Done with HistGradientBoostingClassifier.
Done with HistGradientBoostingClassifier.
Done with HistGradientBoostingClassifier.
Done with HistGradientBoostingClassifier.
Done with LGBMClassifier.
Done with CatBoostClassifier.
CPU times: total: 1.08 s
Wall time: 11min 40s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
3,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.933004,0.900265,0.002224,8 min 8.26 sec
4,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.934512,0.899455,0.002012,2 min 11.36 sec
2,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.958327,0.898667,0.001489,2 min 31.77 sec
7,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.974999,0.898633,0.003085,0 min 50.96 sec
8,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.981378,0.898618,0.003206,1 min 7.91 sec
18,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.939049,0.897724,0.002971,0 min 47.29 sec
16,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.930866,0.897680,0.003052,0 min 6.49 sec
1,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.921078,0.897596,0.001941,0 min 3.32 sec
15,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.942095,0.897519,0.002527,0 min 11.90 sec
19,HistGradientBoostingClassifier,"{'categorical_features': None, 'early_stopping...",0.928550,0.897471,0.002210,0 min 9.77 sec


In [28]:
%%time

# Set seeds for reproducibility
np.random.seed(42)
random.seed(42)

sfs_models = evaluate_models_roc(models, X, y, sfs_features, sk10, f'{experiment_name}_sfs_cat')
sfs_models

Done with CatBoostClassifier.
Done with CatBoostClassifier.
Done with CatBoostClassifier.
Done with CatBoostClassifier.
Done with CatBoostClassifier.
CPU times: total: 78.1 ms
Wall time: 15min 24s


,MLA Name,MLA Parameters,MLA Train ROC AUC,MLA Test ROC AUC,MLA Test ROC AUC Std,MLA Time
4,CatBoostClassifier,"{'iterations': 2500, 'learning_rate': 0.035, '...",0.980475,0.897967,0.001899,3 min 31.66 sec
0,CatBoostClassifier,"{'verbose': False, 'random_state': 5, 'early_s...",0.981920,0.895510,0.002137,2 min 46.36 sec
3,CatBoostClassifier,"{'iterations': 2500, 'learning_rate': 0.025, '...",0.991445,0.895481,0.002223,7 min 52.93 sec
1,CatBoostClassifier,"{'iterations': 2500, 'learning_rate': 0.04, 'l...",0.999508,0.891668,0.001760,10 min 42.27 sec
2,CatBoostClassifier,"{'iterations': 3000, 'learning_rate': 0.065, '...",0.999795,0.888489,0.001065,11 min 49.56 sec


### Post Model Features

In [ ]:
# model1 = LGBMClassifier(n_jobs=-1, random_state=5)
# model2 = XGBClassifier(random_state=5)
# model3 = RandomForestClassifier(random_state=5)
# model4 = ExtraTreesClassifier(random_state=5)
# model5 = HistGradientBoostingClassifier(random_state=5)
# model6 = CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100)

In [32]:
model1 = LGBMClassifier(n_jobs=-1, random_state=5)
model1b = LGBMClassifier(**lgbm_params_1)
model1c = LGBMClassifier(**lgbm_params_2)
model1d = LGBMClassifier(**lgbm_params_3)
model1e = LGBMClassifier(**lgbm_params_4)
model2 = XGBClassifier(random_state=5)
model3 = RandomForestClassifier(random_state=5)
model3b = RandomForestClassifier(**rf_params_1)
model3c = RandomForestClassifier(**rf_params_2)
model4 = ExtraTreesClassifier(random_state=5)
model4b = ExtraTreesClassifier(**extrat_params_1)
model4c = ExtraTreesClassifier(**extrat_params_2)
model4d = ExtraTreesClassifier(**extrat_params_3)
model4e = ExtraTreesClassifier(**extrat_params_4)
model5 = HistGradientBoostingClassifier(random_state=5)
model5b = HistGradientBoostingClassifier(**hist_params_1)
model5c = HistGradientBoostingClassifier(**hist_params_2)
model5d = HistGradientBoostingClassifier(**hist_params_3)
model5e = HistGradientBoostingClassifier(**hist_params_4)
model5f = HistGradientBoostingClassifier(**hist_params_5)
model6 = CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100)
# model6b = CatBoostClassifier(**cat_params_1)
# model6c = CatBoostClassifier(**cat_params_2)
# model6d = CatBoostClassifier(**cat_params_3)
model6e = CatBoostClassifier(**cat_params_4)

- Features for Competition + Original dataset down to SFS for all models (Experiment Set 2)

In [33]:
model1_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index']
model2_feats = ['X_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'Orientation_Index', 'Luminosity_Index']
model3_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index']
model4_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index']
model5_feats = ['X_Minimum', 'Y_Minimum', 'Pixels_Areas', 'Minimum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Orientation_Index', 'Luminosity_Index']
model6_feats = ['X_Minimum', 'Pixels_Areas', 'X_Perimeter', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Outside_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'Orientation_Index', 'Luminosity_Index']

X_lgbm = X[model1_feats]
X_xgb = X[model2_feats]
X_rf = X[model3_feats]
X_extrat = X[model4_feats]
X_hist = X[model5_feats]
X_cat = X[model6_feats]

# Hyperparameter Tuning

- LGBM

In [ ]:
def objective(trial):
    # Raw Parameters for individual tunings
    class_weight_option = trial.suggest_categorical('class_weight', ['none', 'balanced', 'custom'])
    if class_weight_option == 'none':
        class_weight = None
    elif class_weight_option == 'balanced':
        class_weight = 'balanced'
    else:
        # For multi-class, you could define a range or specific values to test
        weight_for_class_0 = trial.suggest_float('weight_for_class_0', 0.1, 10.0)
        weight_for_class_1 = trial.suggest_float('weight_for_class_1', 0.1, 10.0)
        weight_for_class_2 = trial.suggest_float('weight_for_class_2', 0.1, 10.0)
        weight_for_class_3 = trial.suggest_float('weight_for_class_3', 0.1, 10.0)
        weight_for_class_4 = trial.suggest_float('weight_for_class_4', 0.1, 10.0)
        weight_for_class_5 = trial.suggest_float('weight_for_class_5', 0.1, 10.0)
        weight_for_class_6 = trial.suggest_float('weight_for_class_6', 0.1, 10.0)
        class_weight = {0: weight_for_class_0, 1: weight_for_class_1, 2: weight_for_class_2, 3: weight_for_class_3, 4: weight_for_class_4, 5: weight_for_class_5, 6: weight_for_class_6}

    param = {
        'objective': 'multiclass',
        'num_class': 7,
        'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
        # 'class_weight': class_weight,
        # 'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.000001, 0.5),
        # 'max_depth': trial.suggest_int('max_depth', -1, 64),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        # 'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0),
        # 'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 1.0),
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        # 'n_jobs': -1,
        # 'num_leaves': trial.suggest_int('num_leaves', 2, 1000),
        # 'random_state': 5,
        # 'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        # 'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        }

    # # Group Parameters after individual tunings (change after testing the individual params)
    # param = {
    #     'objective': 'multiclass',
    #     'num_class': 7,
    #     'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
    #     'class_weight': class_weight,
    #     'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
    #     'learning_rate': trial.suggest_float('learning_rate', 0.000001, 0.5),
    #     'max_depth': trial.suggest_int('max_depth', -1, 64),
    #     'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
    #     'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0),
    #     'min_split_gain': trial.suggest_float('min_split_gain', 0.5, 1.0),
    #     'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
    #     'n_jobs': -1,
    #     'num_leaves': trial.suggest_int('num_leaves', 2, 1000),
    #     'random_state': 5,
    #     'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
    #     'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
    #     'subsample': trial.suggest_float('subsample', 0.1, 1.0),
    #     }
    
    roc_auc_scores = []
    
    for train_index, test_index in sk10.split(X_lgbm, y):
        X_train, X_test = X_lgbm.iloc[train_index], X_lgbm.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = LGBMClassifier(**param)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)
        preds = model.predict_proba(X_test)
        roc_auc = roc_auc_score(y_test, preds, multi_class='ovr', average='macro')
        roc_auc_scores.append(roc_auc)
    
    return np.mean(roc_auc_scores)

# Using median pruner
pruner = optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=1, interval_steps=1)

study = optuna.create_study(direction='maximize', pruner=pruner)
study.optimize(objective, n_trials=50)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# 0.893695902

- HistGradient

In [ ]:
def objective(trial):
    # # Raw Parameters for individual tunings
    # param = {
    #     # 'learning_rate': trial.suggest_float('learning_rate', 0.000001, 0.15),
    #     # 'max_iter': trial.suggest_int('max_iter', 50, 5000),
    #     # 'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 20, 1000),
    #     # 'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 1000),
    #     # 'l2_regularization': trial.suggest_float('l2_regularization', 0.1, 1.0),
    #     # 'max_bins': trial.suggest_int('max_bins', 10, 255),
    #     'max_depth': trial.suggest_int('max_depth', 2, 64),
    #     'random_state': 5,
    # }

    # Group Parameters after individual tunings (change after testing the individual params)
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.06),
        'max_iter': 100,
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 300, 800),
        'l2_regularization': trial.suggest_float('l2_regularization', 0.8, 1.0),
        'max_bins': trial.suggest_int('max_bins', 50, 150),
        'max_depth': 25,
        'random_state': 5,
    }

    roc_auc_scores = []

    for i, (train_index, test_index) in enumerate(sk10.split(X_hist, y)):
        X_train, X_test = X_hist.iloc[train_index], X_hist.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = HistGradientBoostingClassifier(**param)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_test)
        roc_auc = roc_auc_score(y_test, preds, multi_class='ovr', average='macro')
        roc_auc_scores.append(roc_auc)

        # Report intermediate objective value
        trial.report(roc_auc, i)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

        # # Check if performance is below threshold
        # if roc_auc < performance_threshold:
        #     raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

    return np.mean(roc_auc_scores)

pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)

study = optuna.create_study(direction='maximize', pruner=pruner)
# study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

- ExtraTrees

In [ ]:
def objective(trial):
    # # Raw Parameters for individual tunings
    # param = {
    #     # 'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.1),
    #     # 'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced', 'balanced_subsample']),
    #     # 'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss']),
    #     # 'max_depth': trial.suggest_int('max_depth', 10, 3000),
    #     # 'max_features': trial.suggest_categorical('max_features', [None, 'sqrt', 'log2']),
    #     # 'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 100, 3000, log=True) or None,
    #     # 'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.1),
    #     # 'min_samples_leaf': trial.suggest_int('min_samples_leaf',1, 500),
    #     # 'min_samples_split': trial.suggest_int('min_samples_split', 2, 500),
    #     # 'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5),
    #     'n_estimators': trial.suggest_int('n_estimators', 50, 3000),
    #     'random_state': 5,
    #     'n_jobs': -1,
    # }

    # Group Parameters after individual tunings (change after testing the individual params)
    param = {
        'ccp_alpha': 0.0,
        'class_weight': None,
        'criterion': 'gini',
        'max_depth': None,
        'max_features': 'sqrt',
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 1000, 3000, log=True) or None,
        'min_impurity_decrease': 0.0,
        'min_samples_leaf': trial.suggest_int('min_samples_leaf',1, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 100),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.002),
        'n_estimators': trial.suggest_int('n_estimators', 2500, 3000),
        'random_state': 5,
        'n_jobs': -1,
    }

    roc_auc_scores = []

    for i, (train_index, test_index) in enumerate(sk10.split(X_extrat, y)):
        X_train, X_test = X_extrat.iloc[train_index], X_extrat.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = ExtraTreesClassifier(**param)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_test)
        roc_auc = roc_auc_score(y_test, preds, multi_class='ovr', average='macro')
        roc_auc_scores.append(roc_auc)

        # Report intermediate objective value
        trial.report(roc_auc, i)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

        # # Check if performance is below threshold
        # if roc_auc < performance_threshold:
        #     raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

    return np.mean(roc_auc_scores)

pruner = optuna.pruners.MedianPruner(n_startup_trials=2, n_warmup_steps=0, interval_steps=1)

study = optuna.create_study(direction='maximize', pruner=pruner)
# study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

- RandomForest

In [ ]:
def objective(trial):
    # # # Raw Parameters for individual tunings
    # param = {
    #     # 'ccp_alpha': trial.suggest_float('ccp_alpha', 0.0, 0.1),
    #     # 'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced', 'balanced_subsample']),
    #     # 'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
    #     # 'max_depth': trial.suggest_int('max_depth', 10, 1000, log=True) or None,
    #     # 'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
    #     # 'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 1000, log=True) or None,
    #     # 'max_samples': trial.suggest_float('max_samples', 0.1, 1.0),
    #     # 'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0.0, 0.05),
    #     # 'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 100),
    #     # 'min_samples_split': trial.suggest_int('min_samples_split', 2, 200),
    #     # 'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.1),
    #     'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
    #     'random_state': 5,
    #     'n_jobs': -1,
    # }

    # Group Parameters after individual tunings (change after testing the individual params)
    param = {
        'ccp_alpha': 0.0,
        'class_weight': None,
        'max_depth': trial.suggest_int('max_depth', 10, 30, log=True),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 300, 700, log=True) or None,
        'max_samples': trial.suggest_float('max_samples', 0.1, 1.0),
        'min_impurity_decrease': 0.0,
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_weight_fraction_leaf': 0.0,
        'max_features': 'log2',
        'criterion': 'entropy',
        'n_estimators': 1000,
        'random_state': 5,
        'n_jobs': -1,
    }

    roc_auc_scores = []

    for i, (train_index, test_index) in enumerate(sk10.split(X_rf, y)):
        X_train, X_test = X_rf.iloc[train_index], X_rf.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = RandomForestClassifier(**param)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_test)
        roc_auc = roc_auc_score(y_test, preds, multi_class='ovr', average='macro')
        roc_auc_scores.append(roc_auc)

        # Report intermediate objective value
        trial.report(roc_auc, i)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

        # # Check if performance is below threshold
        # if roc_auc < performance_threshold:
        #     raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

    return np.mean(roc_auc_scores)

pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)

study = optuna.create_study(direction='maximize', pruner=pruner)
# study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

- CatBoost

In [ ]:
def objective(trial):
    # Raw Parameters for individual tunings
    param = {
        'iterations': trial.suggest_int('iterations', 50, 2000),
        # 'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.3, log=True),
        # 'depth': trial.suggest_int('max_depth', 1, 10),
        # 'subsample': trial.suggest_float('subsample', 0.05, 1),
        # 'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
        # 'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        # 'l2_leaf_reg': trial.suggest_float('l2_reg', 1e-2, 10),
        # 'random_strength': trial.suggest_float('random_strength', 1e-2, 10),
        # 'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'random_state': 5,
        'verbose': False,
    }

    # Group Parameters after individual tunings (change after testing the individual params)
    # param = {
    #     'iterations': trial.suggest_int('iterations', 50, 2000),
    #     'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    #     'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.3, log=True),
    #     'depth': trial.suggest_int('max_depth', 1, 10),
    #     'subsample': trial.suggest_float('subsample', 0.05, 1),
    #     'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
    #     'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
    #     'l2_leaf_reg': trial.suggest_float('l2_reg', 1e-2, 10),
    #     'random_strength': trial.suggest_float('random_strength', 1e-2, 10),
    #     'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
    #     'random_state': 5,
    #     'verbose': False,
    # }

    roc_auc_scores = []

    for i, (train_index, test_index) in enumerate(sk10.split(X_rf, y)):
        X_train, X_test = X_rf.iloc[train_index], X_rf.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = CatBoostClassifier(**param)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_test)
        roc_auc = roc_auc_score(y_test, preds, multi_class='ovr', average='macro')
        roc_auc_scores.append(roc_auc)

        # Report intermediate objective value
        trial.report(roc_auc, i)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

        # # Check if performance is below threshold
        # if roc_auc < performance_threshold:
        #     raise optuna.exceptions.TrialPruned('ROC score lower than threshold.')

    return np.mean(roc_auc_scores)

pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1, interval_steps=1)

study = optuna.create_study(direction='maximize', pruner=pruner)
# study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

# Ensembling

In [34]:
%%time

model1_results, model1b_results, model1c_results, model1d_results, model1e_results, model2_results, model3_results, model3b_results, model3c_results, model4_results, model4b_results, model4c_results, model4d_results, model4e_results, model5_results, model5b_results, model5c_results, model5d_results, model5e_results, model5f_results, model6_results, model6b_results, model6c_results, model6d_results, model6e_results, y_test_list = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []


for i, (train_index, test_index) in enumerate(sk10.split(X, y)):
    X_train_lgbm, X_test_lgbm = X_lgbm.iloc[train_index], X_lgbm.iloc[test_index]
    X_train_xgb, X_test_xgb = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    X_train_rf, X_test_rf = X_rf.iloc[train_index], X_rf.iloc[test_index]
    X_train_extrat, X_test_extrat = X_extrat.iloc[train_index], X_extrat.iloc[test_index]
    X_train_hist, X_test_hist = X_hist.iloc[train_index], X_hist.iloc[test_index]
    X_train_cat, X_test_cat = X_cat.iloc[train_index], X_cat.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model1.fit(X_train_lgbm, y_train)
    model1_results.append(model1.predict_proba(X_test_lgbm))

    model1b.fit(X_train_lgbm, y_train)
    model1b_results.append(model1b.predict_proba(X_test_lgbm))

    model1c.fit(X_train_lgbm, y_train)
    model1c_results.append(model1c.predict_proba(X_test_lgbm))

    model1d.fit(X_train_lgbm, y_train)
    model1d_results.append(model1d.predict_proba(X_test_lgbm))

    model1e.fit(X_train_lgbm, y_train)
    model1e_results.append(model1e.predict_proba(X_test_lgbm))

    print('Running XGBoost')

    model2.fit(X_train_xgb, y_train)
    model2_results.append(model2.predict_proba(X_test_xgb))

    print('Running Random Forest')

    model3.fit(X_train_rf, y_train)
    model3_results.append(model3.predict_proba(X_test_rf))

    model3b.fit(X_train_rf, y_train)
    model3b_results.append(model3b.predict_proba(X_test_rf))

    model3c.fit(X_train_rf, y_train)
    model3c_results.append(model3c.predict_proba(X_test_rf))

    print('Running ExtraTrees')

    model4.fit(X_train_extrat, y_train)
    model4_results.append(model4.predict_proba(X_test_extrat))

    model4b.fit(X_train_extrat, y_train)
    model4b_results.append(model4b.predict_proba(X_test_extrat))

    model4c.fit(X_train_extrat, y_train)
    model4c_results.append(model4c.predict_proba(X_test_extrat))

    model4d.fit(X_train_extrat, y_train)
    model4d_results.append(model4d.predict_proba(X_test_extrat))

    model4e.fit(X_train_extrat, y_train)
    model4e_results.append(model4e.predict_proba(X_test_extrat))

    print('Running Hist Gradient')

    model5.fit(X_train_hist, y_train)
    model5_results.append(model5.predict_proba(X_test_hist))

    model5b.fit(X_train_hist, y_train)
    model5b_results.append(model5b.predict_proba(X_test_hist))

    model5c.fit(X_train_hist, y_train)
    model5c_results.append(model5c.predict_proba(X_test_hist))

    model5d.fit(X_train_hist, y_train)
    model5d_results.append(model5d.predict_proba(X_test_hist))

    model5e.fit(X_train_hist, y_train)
    model5e_results.append(model5e.predict_proba(X_test_hist))

    model5f.fit(X_train_hist, y_train)
    model5f_results.append(model5f.predict_proba(X_test_hist))

    print('Running CatBoost')

    model6.fit(X_train_cat, y_train)
    model6_results.append(model6.predict_proba(X_test_cat))

    # model6b.fit(X_train_cat, y_train)
    # model6b_results.append(model6b.predict_proba(X_test_cat))

    # model6c.fit(X_train_cat, y_train)
    # model6c_results.append(model6c.predict_proba(X_test_cat))

    # model6d.fit(X_train_cat, y_train)
    # model6d_results.append(model6d.predict_proba(X_test_cat))

    model6e.fit(X_train_cat, y_train)
    model6e_results.append(model6e.predict_proba(X_test_cat))

    y_test_list.append(y_test)

    print(f'Done with fold {i+1}.')

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
Running XGBoost
Running Random Forest
Running ExtraTrees
Running Hist Gradient
Running CatBoost
Done with fold 1.
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
Running XGBoost
Runnin

In [56]:
%%time

# model1_weights, model1b_weights, model1c_weights, model1d_weights, model1e_weights, model2_weights, model3_weights, model3b_weights, model3c_weights, model4_weights, model4b_weights, model4c_weights, model4d_weights, model4e_weights, model5_weights, model5b_weights, model5c_weights, model5d_weights, model5e_weights, model5f_weights, model6_weights, model6b_weights, model6c_weights, model6d_weights, model6e_weights, scores = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

model1_weights, model1b_weights, model1c_weights, model1d_weights, model1e_weights, model2_weights, model3_weights, model3b_weights, model3c_weights, model4_weights, model4b_weights, model4c_weights, model4d_weights, model4e_weights, model5_weights, model5b_weights, model5c_weights, model5d_weights, model5e_weights, model5f_weights, model6_weights, model6e_weights, scores = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

scores_in = []

for i in tqdm(range(20)):
    weight_1 = np.random.random_sample(size=1)[0]
    weight_1b = np.random.random_sample(size=1)[0]
    weight_1c = np.random.random_sample(size=1)[0]
    weight_1d = np.random.random_sample(size=1)[0]
    weight_1e = np.random.random_sample(size=1)[0]
    weight_2 = np.random.random_sample(size=1)[0]
    weight_3 = np.random.random_sample(size=1)[0]
    weight_3b = np.random.random_sample(size=1)[0]
    weight_3c = np.random.random_sample(size=1)[0]
    weight_4 = np.random.random_sample(size=1)[0]
    weight_4b = np.random.random_sample(size=1)[0]
    weight_4c = np.random.random_sample(size=1)[0]
    weight_4d = np.random.random_sample(size=1)[0]
    weight_4e = np.random.random_sample(size=1)[0]
    weight_5 = np.random.random_sample(size=1)[0]
    weight_5b = np.random.random_sample(size=1)[0]
    weight_5c = np.random.random_sample(size=1)[0]
    weight_5d = np.random.random_sample(size=1)[0]
    weight_5e = np.random.random_sample(size=1)[0]
    weight_5f = np.random.random_sample(size=1)[0]
    weight_6 = np.random.random_sample(size=1)[0]
    # weight_6b = np.random.random_sample(size=1)[0]
    # weight_6c = np.random.random_sample(size=1)[0]
    # weight_6d = np.random.random_sample(size=1)[0]
    weight_6e = np.random.random_sample(size=1)[0]

    model1_weights.append(weight_1)
    model1b_weights.append(weight_1b)
    model1c_weights.append(weight_1c)
    model1d_weights.append(weight_1d)
    model1e_weights.append(weight_1e)
    model2_weights.append(weight_2)
    model3_weights.append(weight_3)
    model3b_weights.append(weight_3b)
    model3c_weights.append(weight_3c)
    model4_weights.append(weight_4)
    model4b_weights.append(weight_4b)
    model4c_weights.append(weight_4c)
    model4d_weights.append(weight_4d)
    model4e_weights.append(weight_4e)
    model5_weights.append(weight_5)
    model5b_weights.append(weight_5b)
    model5c_weights.append(weight_5c)
    model5d_weights.append(weight_5d)
    model5e_weights.append(weight_5e)
    model5f_weights.append(weight_5f)
    model6_weights.append(weight_6)
    # model6b_weights.append(weight_6b)
    # model6c_weights.append(weight_6c)
    # model6d_weights.append(weight_6d)
    model6e_weights.append(weight_6e)

    # scores_in = []

    for j in range(n_splits):
        weighted_pred = (weight_1 * model1_results[j])
        + (weight_1b * model1b_results[j])
        + (weight_1c * model1c_results[j])
        + (weight_1d * model1d_results[j])
        + (weight_1e * model1e_results[j])
        + (weight_2 * model2_results[j])
        + (weight_3 * model3_results[j])
        + (weight_3b * model3b_results[j])
        + (weight_3c * model3c_results[j])
        + (weight_4 * model4_results[j])
        + (weight_4b * model4b_results[j])
        + (weight_4c * model4c_results[j])
        + (weight_4d * model4d_results[j])
        + (weight_4e * model4e_results[j])
        + (weight_5 * model5_results[j])
        + (weight_5b * model5b_results[j])
        + (weight_5c * model5c_results[j])
        + (weight_5d * model5d_results[j])
        + (weight_5e * model5e_results[j])
        + (weight_5f * model5f_results[j])
        + (weight_6 * model6_results[j])
        # + (weight_6b * model6b_results[j])
        # + (weight_6c * model6c_results[j])
        # + (weight_6d * model6d_results[j])
        + (weight_6e * model6e_results[j])

        weighted_pred_normalized = weighted_pred / np.sum(weighted_pred, axis=1, keepdims=True)

        scores_in.append(roc_auc_score(y_test_list[j], weighted_pred_normalized, multi_class='ovr'))
        
    scores.append(np.mean(scores_in))

  0%|          | 0/20 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (6788,7) (6788,) 

In [55]:
results_df = pd.DataFrame()
results_df['model_1'] = model1_weights
results_df['model_1b'] = model1b_weights
results_df['model_1c'] = model1c_weights
results_df['model_1d'] = model1d_weights
results_df['model_1e'] = model1e_weights
results_df['model_2'] = model2_weights
results_df['model_3'] = model3_weights
results_df['model_3b'] = model3b_weights
results_df['model_3c'] = model3c_weights
results_df['model_4'] = model4_weights
results_df['model_4b'] = model4b_weights
results_df['model_4c'] = model4c_weights
results_df['model_4d'] = model4d_weights
results_df['model_4e'] = model4e_weights
results_df['model_5'] = model5_weights
results_df['model_5b'] = model5b_weights
results_df['model_5c'] = model5c_weights
results_df['model_5d'] = model5d_weights
results_df['model_5e'] = model5e_weights
results_df['model_5f'] = model5f_weights
results_df['model_6'] = model6_weights
# results_df['model_6b'] = model6b_weights
# results_df['model_6c'] = model6c_weights
# results_df['model_6d'] = model6d_weights
results_df['model_6e'] = model6e_weights
results_df['score'] = scores

results_df = results_df.sort_values(by='score', ascending=False).reset_index(drop=True)
results_df.head(10)

,model_1,model_1b,model_1c,model_1d,model_1e,model_2,model_3,model_3b,model_3c,model_4,model_4b,model_4c,model_4d,model_4e,model_5,model_5b,model_5c,model_5d,model_5e,model_5f,model_6,model_6e,score
0,0.626877,0.364680,0.100944,0.461033,0.924784,0.583100,0.184504,0.649023,0.465462,0.931380,0.160940,0.712012,0.681602,0.983542,0.564912,0.871574,0.024714,0.122892,0.345939,0.619327,0.449079,0.202696,0.893696
1,0.165247,0.600356,0.232739,0.770631,0.644003,0.571605,0.825227,0.500273,0.269852,0.620705,0.957063,0.601159,0.629619,0.934751,0.515275,0.996357,0.159662,0.537040,0.212901,0.453326,0.170574,0.222529,0.893696
2,0.271119,0.498622,0.438723,0.473713,0.616040,0.230961,0.134704,0.878878,0.633941,0.060663,0.961052,0.693069,0.151705,0.969134,0.365407,0.984510,0.103276,0.159969,0.657420,0.022143,0.018690,0.791983,0.893696
3,0.589204,0.566699,0.524447,0.090556,0.927025,0.592255,0.263992,0.687418,0.317908,0.505856,0.341132,0.880394,0.993981,0.161182,0.023579,0.219323,0.589952,0.173951,0.811484,0.408671,0.610533,0.455618,0.893696
4,0.210800,0.320031,0.134605,0.809342,0.220676,0.416569,0.016057,0.104882,0.896272,0.837160,0.840922,0.411353,0.141531,0.107895,0.605603,0.828193,0.900114,0.288732,0.321027,0.100877,0.774410,0.275690,0.893696
5,0.385890,0.358834,0.148947,0.904053,0.118529,0.472077,0.236518,0.996328,0.585844,0.149585,0.667564,0.991609,0.605043,0.457379,0.293270,0.092123,0.228246,0.553621,0.753786,0.877931,0.793712,0.125465,0.893696
6,0.903328,0.035135,0.729302,0.652172,0.649515,0.413989,0.015675,0.689241,0.829471,0.085230,0.441618,0.162595,0.610938,0.785605,0.666707,0.318060,0.139733,0.794268,0.323917,0.022140,0.571421,0.246677,0.893696
7,0.540048,0.186820,0.526889,0.706941,0.769788,0.523702,0.217368,0.926689,0.773617,0.270688,0.811813,0.347863,0.312431,0.863077,0.873235,0.398481,0.008923,0.186566,0.909296,0.898749,0.453345,0.304117,0.893696
8,0.773460,0.581605,0.469108,0.572764,0.008075,0.067730,0.525849,0.173619,0.545541,0.354214,0.907412,0.949683,0.979784,0.898938,0.985891,0.706147,0.960542,0.641792,0.809214,0.131910,0.682600,0.931103,0.893696
9,0.094016,0.523708,0.506745,0.606548,0.330793,0.925918,0.270061,0.649696,0.495927,0.266673,0.188854,0.956562,0.561291,0.604191,0.782895,0.081227,0.713764,0.165140,0.740481,0.141329,0.832494,0.681016,0.893696


In [40]:
results_df.to_csv('random_weights_normalized.csv', index=False)

# Get Submission (Random Weight Ensemble)

In [39]:
%%time

print('Running LGBM')
model1_final = model1.fit(X_lgbm, y)
model1b_final = model1b.fit(X_lgbm, y)
model1c_final = model1c.fit(X_lgbm, y)
model1d_final = model1d.fit(X_lgbm, y)
model1e_final = model1e.fit(X_lgbm, y)

print('Running XGBoost')
model2_final = model2.fit(X_xgb, y)

print('Running Random Forest')
model3_final = model3.fit(X_rf, y)
model3b_final = model3b.fit(X_rf, y)
model3c_final = model3c.fit(X_rf, y)

print('Running ExtraTrees')
model4_final = model4.fit(X_extrat, y)
model4b_final = model4b.fit(X_extrat, y)
model4c_final = model4c.fit(X_extrat, y)
model4d_final = model4d.fit(X_extrat, y)
model4e_final = model4e.fit(X_extrat, y)

print('Running HistGradient')
model5_final = model5.fit(X_hist, y)
model5b_final = model5b.fit(X_hist, y)
model5c_final = model5c.fit(X_hist, y)
model5d_final = model5d.fit(X_hist, y)
model5e_final = model5e.fit(X_hist, y)
model5f_final = model5f.fit(X_hist, y)

print('Running CatBoost')
model6_final = model6.fit(X_cat, y)
# model6b_final = model6b.fit(X_cat, y)
# model6c_final = model6c.fit(X_cat, y)
# model6d_final = model6d.fit(X_cat, y)
model6e_final = model6e.fit(X_cat, y)

CPU times: total: 22min 41s
Wall time: 16min 20s


In [41]:
%%time

ensemble_pred = (
                results_df['model_1'][0] * model1_final.predict_proba(test[model1_feats]) +
                results_df['model_1b'][0] * model1b_final.predict_proba(test[model1_feats]) +
                results_df['model_1c'][0] * model1c_final.predict_proba(test[model1_feats]) +
                results_df['model_1d'][0] * model1d_final.predict_proba(test[model1_feats]) +
                results_df['model_1e'][0] * model1e_final.predict_proba(test[model1_feats]) +
                results_df['model_2'][0] * model2_final.predict_proba(test[model2_feats]) +
                results_df['model_3'][0] * model3_final.predict_proba(test[model3_feats]) +
                results_df['model_3b'][0] * model3b_final.predict_proba(test[model3_feats]) +
                results_df['model_3c'][0] * model3c_final.predict_proba(test[model3_feats]) +
                results_df['model_4'][0] * model4_final.predict_proba(test[model4_feats]) +
                results_df['model_4b'][0] * model4b_final.predict_proba(test[model4_feats]) +
                results_df['model_4c'][0] * model4c_final.predict_proba(test[model4_feats]) +
                results_df['model_4d'][0] * model4d_final.predict_proba(test[model4_feats]) +
                results_df['model_4e'][0] * model4e_final.predict_proba(test[model4_feats]) +
                results_df['model_5'][0] * model5_final.predict_proba(test[model5_feats]) +
                results_df['model_5b'][0] * model5b_final.predict_proba(test[model5_feats]) +
                results_df['model_5c'][0] * model5c_final.predict_proba(test[model5_feats]) +
                results_df['model_5d'][0] * model5d_final.predict_proba(test[model5_feats]) +
                results_df['model_5e'][0] * model5e_final.predict_proba(test[model5_feats]) +
                results_df['model_5f'][0] * model5f_final.predict_proba(test[model5_feats]) +
                results_df['model_6'][0] * model6_final.predict_proba(test[model6_feats]) +
                # results_df['model_6b'][0] * model6b_final.predict_proba(test[model6_feats]) +
                # results_df['model_6c'][0] * model6c_final.predict_proba(test[model6_feats]) +
                # results_df['model_6d'][0] * model6d_final.predict_proba(test[model6_feats]) +
                results_df['model_6e'][0] * model6e_final.predict_proba(test[model6_feats])
                 )

ensemble_df = pd.DataFrame(ensemble_pred)

# If all models predict 0, instead of getting NaN, fill in 0
ensemble_df = ensemble_df.div(ensemble_df.sum(axis=1), axis=0).fillna(0)
ensemble_df.columns = label_encoder.classes_

In [42]:
ensemble_df.head()

,Bumps,Dirtiness,K_Scatch,Other_Faults,Pastry,Stains,Z_Scratch
0,0.131240,0.026219,0.005702,0.343930,0.488380,0.000387,0.004143
1,0.154362,0.211441,0.009527,0.345254,0.258984,0.001850,0.018583
2,0.276297,0.009285,0.067533,0.592444,0.004124,0.007742,0.042575
3,0.390152,0.012591,0.000884,0.446294,0.140416,0.002809,0.006854
4,0.608551,0.009964,0.002360,0.358683,0.008212,0.005592,0.006637


In [43]:
submission = pd.read_csv('sample_submission.csv')
submission_df = pd.concat([submission['id'], ensemble_df], axis=1)
submission_df.head()

,id,Bumps,Dirtiness,K_Scatch,Other_Faults,Pastry,Stains,Z_Scratch
0,19219,0.131240,0.026219,0.005702,0.343930,0.488380,0.000387,0.004143
1,19220,0.154362,0.211441,0.009527,0.345254,0.258984,0.001850,0.018583
2,19221,0.276297,0.009285,0.067533,0.592444,0.004124,0.007742,0.042575
3,19222,0.390152,0.012591,0.000884,0.446294,0.140416,0.002809,0.006854
4,19223,0.608551,0.009964,0.002360,0.358683,0.008212,0.005592,0.006637


In [44]:
submission_df.to_csv('submission_ensemble_3fold_0.893696.csv', index=False)

# Get submission (Stacking)

In [ ]:
%%time

from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import numpy as np


roc_auc_scores = []

# Define the base models
base_models = [
    ('model1', model1_final),
    ('model2', model2_final),
    ('model3', model3_final),
    ('model4', model4_final),
    ('model5', model5_final),
    ('model6', model6_final)
]

# Initialize the Stacking Classifier with LogisticRegression as the final estimator
final_estimator = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
# final_estimator = LGBMClassifier(n_jobs=-1, random_state=5)
# final_estimator = XGBClassifier(random_state=5)
# final_estimator = RandomForestClassifier(random_state=5)
# final_estimator = ExtraTreesClassifier(random_state=5)
# final_estimator = HistGradientBoostingClassifier(random_state=5)
# final_estimator = CatBoostClassifier(random_state=5, verbose=False, early_stopping_rounds=100)

stacking_clf = StackingClassifier(estimators=base_models, final_estimator=final_estimator, passthrough=False, cv=3)

for i, (train_index, test_index) in enumerate(sk10.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    stacking_clf.fit(X_train, y_train)
    y_pred = stacking_clf.predict_proba(X_test)

    # Assuming your classes are 0, 1, 2, etc., adjust as necessary
    y_test_binarized = label_binarize(y_test, classes=np.unique(y))
    roc_auc = roc_auc_score(y_test_binarized, y_pred, multi_class='ovr')

    roc_auc_scores.append(roc_auc)

    print(f'Done with fold {i+1}.')
    
print(f'The average stacking score is {np.mean(roc_auc_scores)}')

- Logistic Reg - 0.886778
- LGBM - 0.885863
- XGB - 0.881636
- RF - 0.883835
- ET - 0.884523
- Hist - 0.886572
- Cat - 0.886183

In [ ]:
# Predictions on unseen test data
y_test_pred = stacking_clf.predict_proba(test)

stacking_df = pd.DataFrame(y_test_pred)

ensemble_df.head()

In [ ]:
%%time

model1_results, model2_results, model3_results, model4_results, model5_results, model6_results, y_test_list = [], [], [], [], [], [], []

# # Placeholder for OOF predictions for each model
# # Assuming you have a dataset with N samples
# N = len(y)  # y_train is your target variable array
# oof_preds1 = np.zeros((N, 1))
# oof_preds2 = np.zeros((N, 1))
# oof_preds3 = np.zeros((N, 1))
# oof_preds4 = np.zeros((N, 1))
# oof_preds5 = np.zeros((N, 1))
# oof_preds6 = np.zeros((N, 1))

# # Similarly, for test predictions, accumulate them over folds
# # Assuming you have a test set with M samples
# M = len(test)  # x_test needs to be defined by you
# test_preds1 = np.zeros((M, 1))
# test_preds2 = np.zeros((M, 1))
# test_preds3 = np.zeros((M, 1))
# test_preds4 = np.zeros((M, 1))
# test_preds5 = np.zeros((M, 1))
# test_preds6 = np.zeros((M, 1))

target_length = len(y)
no_classes = len(np.unique(y))
test_length = len(test)

# Initialize arrays for OOF and test predictions with dimensions for multiclass for each model
lgbm_oof_preds = np.zeros((target_length, no_classes))
lgbm_test_preds = np.zeros((test_length, no_classes))

xgb_oof_preds = np.zeros((target_length, no_classes))
xgb_test_preds = np.zeros((test_length, no_classes))

rf_oof_preds = np.zeros((target_length, no_classes))
rf_test_preds = np.zeros((test_length, no_classes))

extrat_oof_preds = np.zeros((target_length, no_classes))
extrat_test_preds = np.zeros((test_length, no_classes))

hist_oof_preds = np.zeros((target_length, no_classes))
hist_test_preds = np.zeros((test_length, no_classes))

cat_oof_preds = np.zeros((target_length, no_classes))
cat_test_preds = np.zeros((test_length, no_classes))

X_lgbm = X[model1_feats]
X_xgb = X[model2_feats]
X_rf = X[model3_feats]
X_extrat = X[model4_feats]
X_hist = X[model5_feats]
X_cat = X[model6_feats]


In [ ]:

for i, (train_index, test_index) in enumerate(sk10.split(X, y)):

    # Placeholder arrays for the fold's predicition
    fold_oof_preds_lgbm = np.zeros((len(test_index), no_classes))
    fold_test_preds_lgbm = np.zeros((test_length, no_classes))

    fold_oof_preds_xgb = np.zeros((len(test_index), no_classes))
    fold_test_preds_xgb = np.zeros((test_length, no_classes))

    fold_oof_preds_rf = np.zeros((len(test_index), no_classes))
    fold_test_preds_rf = np.zeros((test_length, no_classes))

    fold_oof_preds_extrat = np.zeros((len(test_index), no_classes))
    fold_test_preds_extrat = np.zeros((test_length, no_classes))

    fold_oof_preds_hist = np.zeros((len(test_index), no_classes))
    fold_test_preds_hist = np.zeros((test_length, no_classes))

    fold_oof_preds_cat = np.zeros((len(test_index), no_classes))
    fold_test_preds_cat = np.zeros((test_length, no_classes))

    # Get each models train and test for X and y
    X_train_lgbm, X_test_lgbm = X_lgbm.iloc[train_index], X_lgbm.iloc[test_index]
    X_train_xgb, X_test_xgb = X_xgb.iloc[train_index], X_xgb.iloc[test_index]
    X_train_rf, X_test_rf = X_rf.iloc[train_index], X_rf.iloc[test_index]
    X_train_extrat, X_test_extrat = X_extrat.iloc[train_index], X_extrat.iloc[test_index]
    X_train_hist, X_test_hist = X_hist.iloc[train_index], X_hist.iloc[test_index]
    X_train_cat, X_test_cat = X_cat.iloc[train_index], X_cat.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ########
    # LGBM #
    ########
    model1.fit(X_train_lgbm, y_train)
    fold_oof_preds_lgbm = model1.predict_proba(X_test_lgbm)

    # Update the OOF prediction for this fold
    lgbm_oof_preds[test_index] = fold_oof_preds_lgbm

    # Predict on the test set and accumulate predictions
    fold_test_preds_lgbm += model1.predict_proba(test.loc[:, model1_feats]) / sk10.n_splits

    lgbm_test_preds += fold_test_preds_lgbm


    ###########
    # XGBOOST #
    ###########
    model2.fit(X_train_xgb, y_train)
    fold_oof_preds_xgb = model2.predict_proba(X_test_xgb)

    # Update the OOF prediction for this fold
    xgb_oof_preds[test_index] = fold_oof_preds_xgb

    # Predict on the test set and accumulate predictions
    fold_test_preds_xgb += model2.predict_proba(test.loc[:, model2_feats]) / sk10.n_splits

    xgb_test_preds += fold_test_preds_xgb


    #################
    # RANDOM FOREST #
    #################
    model3.fit(X_train_rf, y_train)
    fold_oof_preds_rf = model3.predict_proba(X_test_rf)

    # Update the OOF prediction for this fold
    rf_oof_preds[test_index] = fold_oof_preds_rf

    # Predict on the test set and accumulate predictions
    fold_test_preds_rf += model3.predict_proba(test.loc[:, model3_feats]) / sk10.n_splits

    rf_test_preds += fold_test_preds_rf

    
    ###############
    # EXTRA TREES #
    ###############
    model4.fit(X_train_extrat, y_train)
    fold_oof_preds_extrat = model4.predict_proba(X_test_extrat)

    # Update the OOF prediction for this fold
    extrat_oof_preds[test_index] = fold_oof_preds_extrat

    # Predict on the test set and accumulate predictions
    fold_test_preds_extrat += model4.predict_proba(test.loc[:, model4_feats]) / sk10.n_splits

    extrat_test_preds += fold_test_preds_extrat


    #################
    # HIST GRADIENT #
    #################
    model5.fit(X_train_hist, y_train)
    fold_oof_preds_hist = model5.predict_proba(X_test_hist)

    # Update the OOF prediction for this fold
    hist_oof_preds[test_index] = fold_oof_preds_hist

    # Predict on the test set and accumulate predictions
    fold_test_preds_hist += model5.predict_proba(test.loc[:, model5_feats]) / sk10.n_splits

    hist_test_preds += fold_test_preds_hist


    ############
    # CATBOOST #
    ############
    model6.fit(X_train_cat, y_train)
    fold_oof_preds_cat = model6.predict_proba(X_test_cat)

    # Update the OOF prediction for this fold
    cat_oof_preds[test_index] = fold_oof_preds_cat

    # Predict on the test set and accumulate predictions
    fold_test_preds_cat += model6.predict_proba(test.loc[:, model6_feats]) / sk10.n_splits

    cat_test_preds += fold_test_preds_cat
    # y_test_list.append(y_test)

    print(f'Done with fold {i+1}.')

In [ ]:
# roc_auc_scores = [roc_auc_score((y == class_id).astype(int), oof_preds[:, class_id], multi_class='ovr') for class_id in range(no_classes)]
lgbm_roc_auc = roc_auc_score(y, lgbm_oof_preds, multi_class='ovr', average='macro')
print("Average LGBM ROC AUC Score:", lgbm_roc_auc)

xgb_roc_auc = roc_auc_score(y, xgb_oof_preds, multi_class='ovr', average='macro')
print("Average XGBoost ROC AUC Score:", xgb_roc_auc)

rf_roc_auc = roc_auc_score(y, rf_oof_preds, multi_class='ovr', average='macro')
print("Average Random Forest ROC AUC Score:", rf_roc_auc)

extrat_roc_auc = roc_auc_score(y, extrat_oof_preds, multi_class='ovr', average='macro')
print("Average Extra Trees ROC AUC Score:", extrat_roc_auc)

hist_roc_auc = roc_auc_score(y, hist_oof_preds, multi_class='ovr', average='macro')
print("Average Hist Gradient ROC AUC Score:", hist_roc_auc)

cat_roc_auc = roc_auc_score(y, cat_oof_preds, multi_class='ovr', average='macro')
print("Average CatBoost ROC AUC Score:", cat_roc_auc)

# 0.89369590207664
# 0.00201442835387733
# 0.886778 - StackingClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression

# After running the fitting and prediction with the first level of machine learning models
x_train = np.concatenate(( lgbm_oof_preds, xgb_oof_preds, rf_oof_preds, extrat_oof_preds, hist_oof_preds, cat_oof_preds), axis=1)
test_stack = np.concatenate(( lgbm_test_preds, xgb_test_preds, rf_test_preds, extrat_test_preds, hist_test_preds, cat_test_preds), axis=1)

# Assuming the second-level stacking is to be done with XGboost (pre-tuned). Yes! You can tune second-level stack

stacking_estimator = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')

xgb = stacking_estimator.fit(x_train, y)
final_predictions = xgb.predict_proba(test_stack)

In [ ]:
oof_preds = np.zeros((x_train.shape[0], no_classes))
test_preds = np.zeros(test_stack.shape[0])

for i, (train_index, test_index) in enumerate(sk10.split(x_train, y)):
    X_train, X_test = x_train[train_index], x_train[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model2.fit(X_train, y_train)
    y_pred = model2.predict_proba(X_test)

    # Assign predictions for this fold to the appropriate indices in oof_preds
    oof_preds[test_index, :] = y_pred
    
    print(f'Done with fold {i+1}.')

# Calculate ROC AUC on the OOF predictions
roc_auc = roc_auc_score(y, oof_preds, multi_class='ovr', average='macro')
print(f'The stacking score is {roc_auc}')

- Logistic Reg - 0.8883102077923056
- LGBM - 0.8880225088607244
- XGB - 0.8846028966376445
- RF - 
- ET - 
- Hist - 
- Cat - 

In [ ]:
final_predictions_df = pd.DataFrame(final_predictions)
final_predictions_df.columns = label_encoder.classes_

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission_df = pd.concat([submission['id'], final_predictions_df], axis=1)
submission_df.head()

In [ ]:
submission_df.to_csv('submission_stacking_3fold_0.88831.csv', index=False)